## **Hasta Ejercicio 2.6**

In [1]:
from kafka import KafkaProducer
import json
import time
import random
from math import cos, pi
import numpy as np

In [2]:
def generate_sensor_data():
    timestamp = int(time.time())

    # Simulate IoT sensor data for water quality metrics with realistic patterns
    water_temperature = random.uniform(1, 3) + round(20 + 5 * (1 + 0.5 * (1 + cos((timestamp % 86400) / 86400 * 2 * pi))), 2)
    ph_level = random.uniform(0, 1) + round(7.5 + 0.2 * (1 + cos((timestamp % 86400) / 86400 * 2 * pi)), 2)
    turbidity = round(random.uniform(5, 50), 2)  # Turbidity in NTU (Nephelometric Turbidity Units)
    dissolved_oxygen = round(random.uniform(5, 12), 2)  # Dissolved Oxygen in mg/L
    
    return {
        "timestamp": timestamp,
        "water_temperature": water_temperature,
        "ph_level": ph_level,
        "turbidity": turbidity,
        "dissolved_oxygen": dissolved_oxygen
    }


In [5]:
import sys
import time
# Kafka configuration
kafka_topic = "water_quality"
kafka_bootstrap_servers = ["localhost:9092"] 

producer_number = 3
producers = []
producer_ids = []

for i in range(producer_number):
    producer_ids.append(str(i))
    # Create Kafka producer
    producer = KafkaProducer(bootstrap_servers=kafka_bootstrap_servers,value_serializer=lambda v: json.dumps(v).encode('utf-8'))

    producers.append(producer)

print(f"Producing messages to Kafka topic '{kafka_topic}'...")

try:
    while True:
        for i in range(producer_number):
            # Generate sensor data
            sensor_data = generate_sensor_data()
            print(f'Productor: {i}')
            # Publish sensor data to Kafka
            producers[i].send(kafka_topic, key=producer_ids[i].encode('utf-8'),value=sensor_data)
            print(f"Sent: {sensor_data}")

        # Wait for 1 second
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopped producing messages.")
finally:
    for producer in producers:
        producer.close()

Producing messages to Kafka topic 'water_quality'...
Productor: 0
Sent: {'timestamp': 1741174738, 'water_temperature': 26.76680042699825, 'ph_level': 7.917129207297975, 'turbidity': 29.63, 'dissolved_oxygen': 6.35}
Productor: 1
Sent: {'timestamp': 1741174738, 'water_temperature': 27.4938983241883, 'ph_level': 7.744088351841525, 'turbidity': 19.06, 'dissolved_oxygen': 8.56}
Productor: 2
Sent: {'timestamp': 1741174739, 'water_temperature': 26.35744692525732, 'ph_level': 7.853927959346652, 'turbidity': 13.21, 'dissolved_oxygen': 10.39}
Productor: 0
Sent: {'timestamp': 1741174740, 'water_temperature': 27.71728098358934, 'ph_level': 7.524504494228858, 'turbidity': 38.8, 'dissolved_oxygen': 10.37}
Productor: 1
Sent: {'timestamp': 1741174740, 'water_temperature': 26.764534648637614, 'ph_level': 8.167764383959064, 'turbidity': 35.17, 'dissolved_oxygen': 8.15}
Productor: 2
Sent: {'timestamp': 1741174740, 'water_temperature': 27.855342288764806, 'ph_level': 7.570903713470159, 'turbidity': 17.71,

## **Ejercicio 2.7**


**Enunciado**: Modifica el archivo water_data_consumer_pr_2.ipynb para tener consumidores con un group_id común. Cada sensor_id debe producir un plot con los datos producidos por ese sensor_id. Crea una copia de water_data_consumer_pr_2.ipynb modificado y adjunta captura de los dos notebooks modificados ejecutándose a la vez y gestionando diferentes sensor_id y los plots resultantes.

Si se debe generar un gráfico (plot) por cada sensor, entonces habrá tantos productores como sensores. Supongamos que tenemos 3 sensores; en ese caso, necesitamos implementar 3 productores, los cuales escribirán en un mismo topic, llamado 'water_quality'. Dado que cada consumidor debe procesar los datos de un sensor específico para generar un gráfico, es necesario dividir el topic en 3 particiones. De esta forma, cada consumidor podrá acceder a una partición que esté asociada a un sensor particular.



In [7]:
from kafka import KafkaProducer
from kafka import KafkaAdminClient
from kafka.admin import NewTopic

import json
import time
import random
from math import cos, pi
import numpy as np

In [8]:
def generate_sensor_data():
    timestamp = int(time.time())

    # Simulate IoT sensor data for water quality metrics with realistic patterns
    water_temperature = random.uniform(1, 3) + round(20 + 5 * (1 + 0.5 * (1 + cos((timestamp % 86400) / 86400 * 2 * pi))), 2)
    ph_level = random.uniform(0, 1) + round(7.5 + 0.2 * (1 + cos((timestamp % 86400) / 86400 * 2 * pi)), 2)
    turbidity = round(random.uniform(5, 50), 2)  # Turbidity in NTU (Nephelometric Turbidity Units)
    dissolved_oxygen = round(random.uniform(5, 12), 2)  # Dissolved Oxygen in mg/L
    
    return {
        "timestamp": timestamp,
        "water_temperature": water_temperature,
        "ph_level": ph_level,
        "turbidity": turbidity,
        "dissolved_oxygen": dissolved_oxygen
    }


In [12]:
### CÓDIGO PARA ELIMINAR EL TOPIC Y VOLVER A CREARLO CON "N" PARTICIONES
# admin_client = KafkaAdminClient(bootstrap_servers=kafka_bootstrap_servers)
# existing_topics = admin_client.list_topics()

# if kafka_topic in existing_topics:
#     print(f"El topic '{kafka_topic}' ya existe. Eliminándolo...")
#     admin_client.delete_topics([kafka_topic])

El topic 'water_quality' ya existe. Eliminándolo...


In [13]:
### Creacion del topic con tantas particiones como sensores existan.
sensors = 2
# Se usa el cliente de Kafka en Python
admin_client = KafkaAdminClient(bootstrap_servers=kafka_bootstrap_servers)
topic = NewTopic(name='water_quality', num_partitions=sensors, replication_factor=1)

admin_client.create_topics([topic])

print(f"Topic '{kafka_topic}' creado con {sensors} particiones.")

Topic 'water_quality' creado con 2 particiones.


In [ ]:
import sys
import time
# Kafka configuration
kafka_topic = "water_quality"
kafka_bootstrap_servers = ["localhost:9092"] 

# 2 sensores
producer_number = sensors
producers = []
producer_ids = []

for i in range(producer_number):
    producer_ids.append(str(i))
    # Create Kafka producer
    producer = KafkaProducer(bootstrap_servers=kafka_bootstrap_servers,value_serializer=lambda v: json.dumps(v).encode('utf-8'))
    producers.append(producer)

print(f"Producing messages to Kafka topic '{kafka_topic}'...")

try:
    while True:
        for i in range(producer_number):
            # Generate sensor data
            sensor_data = generate_sensor_data()
            print(f'Sensor: {i} sends data: {sensor_data}')
            # Publish sensor data to Kafka
            producers[i].send(kafka_topic, key=producer_ids[i].encode('utf-8'),value=sensor_data)

        time.sleep(1)
except KeyboardInterrupt:
    print("Stopped producing messages.")
finally:
    for producer in producers:
        producer.close()